# Running on test dataset

Date: *Jun 22*

In [ ]:
import pandas as pd

TASKNAMES = ["subtask1", "subtask2"]
TYPES = ["test"]

dataset_loader = dict()

for taskname in TASKNAMES:
    for type in TYPES:
        print(f"Processing {taskname} for {type}")
        prefix = "dataset/"
        # Load the data
        
        data = pd.read_csv(f"{prefix}/{taskname}_{type}.csv")
        dataset_loader[f"{taskname}_{type}"] = data

In [ ]:
dataset_loader["subtask1_test"] == dataset_loader["subtask2_test"]

# Preprocessing the data

Pronouns -> Named Entity

In [ ]:
from geminillm import gemini_complete_if_cache
from json_kv_iml import JsonKVStorage
from operationCheatSheet import preprocessing_claim
from openaillm import openai_embed

LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20"


kv_global_config = {
    "working_dir": "/tmp",
    "llm_model_name": LLM_MODEL_NAME,
    "embedding_batch_num": 64,  # or another integer suitable for your setup
    "vector_db_storage_cls_kwargs": {
        "cosine_better_than_threshold": 0.2  # or another float threshold you want
    },
    "base_url": "https://api.openai.com/v1",
}

llm_cache = JsonKVStorage(namespace="llm_cache", global_config=kv_global_config, embedding_func=openai_embed)
    
async def llm_wrapper(prompt, history_messages=None, max_tokens=None, **kwargs):
    if history_messages is None:
        history_messages = []

    # Use Google GenAI
    return await gemini_complete_if_cache(
        model=LLM_MODEL_NAME,
        prompt=prompt,
        history_messages=history_messages,
        hashing_kv=llm_cache,
        temperature=0.2,
        max_tokens=max_tokens or 1024,
    )
answer = dataset_loader["subtask1_test"].answer.tolist()[0]
claim = dataset_loader["subtask1_test"].claim.tolist()[0]
response = await preprocessing_claim(claim, answer, llm_wrapper, llm_cache, 1024, [])
response

In [ ]:
import os

all_answer_test = dataset_loader["subtask1_test"].answer.tolist()
all_claim_test = dataset_loader["subtask1_test"].claim.tolist()

preprocess_path = "dataset/subtask1_test_preprocess.csv"
if os.path.exists(preprocess_path):
    df = pd.read_csv(preprocess_path)
else:
    # Create an empty DataFrame if file doesn't exist
    df = pd.DataFrame()
for index, (answer, claim) in enumerate(zip(all_answer_test, all_claim_test)):
    if index % 200 == 0:
        print(f"Processing index {index}")
    try:
        response = await preprocessing_claim(claim, answer, llm_wrapper, llm_cache, 1024, [])
        if response is None:
            print(f"Index {index} has no response")
            continue
        else:
            df.at[index, "claim"] = response
    except Exception as e:
        print(f"Index {index} has error")
        print(e)

df.to_csv(preprocess_path, index=False)

# Load the processed data

In [1]:
import pandas as pd

from geminillm import gemini_complete_if_cache
from json_kv_iml import JsonKVStorage
from operationCheatSheet import preprocessing_claim
from openaillm import openai_embed

LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20"

TASKNAMES = ["subtask1", "subtask2"]
TYPES = ["test", "test_preprocess"]

dataset_loader = dict()

for taskname in TASKNAMES:
    for type in TYPES:
        print(f"Processing {taskname} for {type}")
        prefix = "dataset"
        # Load the data
        
        if taskname == "subtask2" and type == "test_preprocess":
            continue
        data = pd.read_csv(f"{prefix}/{taskname}_{type}.csv")
        dataset_loader[f"{taskname}_{type}"] = data

dataset_loader["subtask1_test"]["claim"] = dataset_loader["subtask1_test_preprocess"]["claim"]

Processing subtask1 for test
Processing subtask1 for test_preprocess
Processing subtask2 for test
Processing subtask2 for test_preprocess


## Run entity recognition on claims

In [2]:
import asyncio
import os
import networkx as nx

from dotenv import load_dotenv
from operationCheatSheet import chunking_by_token_size, extract_entities, extract_evidence
from utils import compute_mdhash_id
from geminillm import gemini_complete_if_cache
from openaillm import openai_embed, openai_complete
from faiss_impl import FaissVectorDBStorage
from networkx_impl import NetworkXStorage
from json_kv_iml import JsonKVStorage
from shared_storage import initialize_share_data
from cheatsheet import CHEATSHEETS


# LLM_MODEL_NAME = "gpt-3.5-turbo"
LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20"

load_dotenv()

async def process_text_to_knowledge_graph_on_claim(text_content,
                                          cheatsheet_knowledge_graph_inst,
                                          max_gleaning = 1,
                                          file_path="unknown_source",
                                          prefix=""):
    # Step 1: Create chunks from your text content
    raw_chunks = chunking_by_token_size(
        content=text_content,
        split_by_character=None,  # Optional: specify a character to split by (e.g., "\n\n")
        split_by_character_only=False,
        overlap_token_size=128,
        max_token_size=1024,
        tiktoken_model=LLM_MODEL_NAME
    )
    
    # Step 2: Format chunks as required by extract_entities
    doc_id = compute_mdhash_id(text_content, prefix="doc-")
    chunks = {
        compute_mdhash_id(chunk["content"], prefix="chunk-"): {
            **chunk,
            "full_doc_id": doc_id,
            "file_path": file_path
        }
        for chunk in raw_chunks
    }
    
    # Step 3: Initialize your storage instances
    faiss_global_config = {
        "working_dir": "/tmp",
        "embedding_batch_num": 64,  # or another integer suitable for your setup
        "vector_db_storage_cls_kwargs": {
            "cosine_better_than_threshold": 0.2  # or another float threshold you want
        },
        "base_url": "https://api.openai.com/v1",
    }
    kv_global_config = {
        "working_dir": "/tmp",
        "llm_model_name": LLM_MODEL_NAME,
        "embedding_batch_num": 64,  # or another integer suitable for your setup
        "vector_db_storage_cls_kwargs": {
            "cosine_better_than_threshold": 0.2  # or another float threshold you want
        },
        "base_url": "https://api.openai.com/v1",
    }
    knowledge_graph = NetworkXStorage(namespace="nx_kg", global_config=faiss_global_config, embedding_func=openai_embed)  # Graph storage implementation
    entity_vector_db = FaissVectorDBStorage(namespace="faiss_entity", global_config=faiss_global_config, embedding_func=openai_embed)  # Vector storage for entities
    relationship_vector_db = FaissVectorDBStorage(namespace="faiss_relationship", global_config=faiss_global_config, embedding_func=openai_embed)  # Vector storage for relationships
    llm_cache = JsonKVStorage(namespace="llm_cache", global_config=kv_global_config, embedding_func=openai_embed)
    
    initialize_share_data()

    await knowledge_graph.initialize()
    await entity_vector_db.initialize()
    await relationship_vector_db.initialize()
    await llm_cache.initialize()  # if your cache supports/needs it

    # Step 4: Use the OpenAI LLM function from openai.py
    # Create a wrapper that matches the expected signature for extract_entities
    async def llm_wrapper(prompt, history_messages=None, max_tokens=None, **kwargs):
        if history_messages is None:
            history_messages = []
    

        # Use Google GenAI
        return await gemini_complete_if_cache(
            model=LLM_MODEL_NAME,
            prompt=prompt,
            history_messages=history_messages,
            hashing_kv=llm_cache,
            temperature=0.2,
            max_tokens=max_tokens or 1024,
        )
    
    # Step 5: Configure the extraction process
    global_config = {
        "llm_model_func": llm_wrapper,  # Use our wrapper function
        "llm_model_name": LLM_MODEL_NAME,  # Required for some OpenAI functions
        "entity_extract_max_gleaning": max_gleaning,
        "force_llm_summary_on_merge": False,
        "llm_model_max_token_size": 1024,
        "summary_to_max_tokens": 256,
        "tiktoken_model_name": LLM_MODEL_NAME,
        "addon_params": {
            "language": "English",
            "entity_types": [
                "Software or Computational Method",
                "Scientific Method",
                "Infrastructure or Equipment",
                "Institution or Organization",
                "Ecological or Biological Entity",
                "Pollutant or Material",
                "Time Expression",
                "Health or Disease Concept",
                "Food or Nutrition Element",
                "Demographic Group",
                "Measurement or Quantity"
            ]
        },
    }
    
    # Step 6: Optional status tracking
    pipeline_status = {
        "latest_message": "",
        "history_messages": []
    }
    pipeline_status_lock = asyncio.Lock()
    
    # Step 7: Run the extraction
    await extract_entities(
        chunks=chunks,
        knowledge_graph_inst=knowledge_graph,
        entity_vdb=entity_vector_db,
        relationships_vdb=relationship_vector_db,
        global_config=global_config,
        pipeline_status=pipeline_status,
        pipeline_status_lock=pipeline_status_lock,
        llm_response_cache=llm_cache,
        cheatsheet_knowledge_graph_inst=cheatsheet_knowledge_graph_inst,
        write_result_to_txt=True,
        special_interest=CHEATSHEETS["special_interests"],
        prefix=prefix
    )
    
    return knowledge_graph


# Example usage in a Jupyter notebook
async def run_on_claim(content, nx_graph, max_gleaning, prefix=""):
    # monkeyReader = reader.MonkeyReader('monkey') # or 'lxml' or 'x2d'

    # read paper content
    # essay = monkeyReader.readEssay('./xmldata/Byjesh.pdf.tei.xml')
    # text = "\n".join(essay["Impact of fixed rise in temperature, CO 2 and change in rain fall"])
    text = content
    # nx_graph = nx.Graph()
    knowledge_graph = await process_text_to_knowledge_graph_on_claim(text, nx_graph,  max_gleaning, "example.txt", prefix=prefix)
    # Now you can query the knowledge graph for entities and relationships
    
    return knowledge_graph

In [ ]:
from utils import get_latest_result
from tqdm import trange
test_data = dataset_loader["subtask1_test"]
for index in trange(len(test_data)):
    claim_content = test_data.at[index, "claim"]

    _ = await run_on_claim(claim_content, nx.Graph(),
                           0, prefix=f"claim_batch2_{index}")

# Run on Reference

Base on claim results run entity extraction on reference

In [3]:
async def process_text_to_knowledge_graph_on_reference(text_content,
                                          cheatsheet_knowledge_graph_inst,
                                          seed_entities,
                                          max_gleaning = 1,
                                          file_path="unknown_source",
                                          prefix=""
                                          ):
    # Step 1: Create chunks from your text content
    raw_chunks = chunking_by_token_size(
        content=text_content,
        split_by_character=None,  # Optional: specify a character to split by (e.g., "\n\n")
        split_by_character_only=False,
        overlap_token_size=128,
        max_token_size=1024,
        tiktoken_model=LLM_MODEL_NAME
    )
    
    # Step 2: Format chunks as required by extract_entities
    doc_id = compute_mdhash_id(text_content, prefix="doc-")
    chunks = {
        compute_mdhash_id(chunk["content"], prefix="chunk-"): {
            **chunk,
            "full_doc_id": doc_id,
            "file_path": file_path
        }
        for chunk in raw_chunks
    }
    
    # Step 3: Initialize your storage instances
    faiss_global_config = {
        "working_dir": "/tmp",
        "embedding_batch_num": 64,  # or another integer suitable for your setup
        "vector_db_storage_cls_kwargs": {
            "cosine_better_than_threshold": 0.2  # or another float threshold you want
        },
        "base_url": "https://api.openai.com/v1",
    }
    kv_global_config = {
        "working_dir": "/tmp",
        "llm_model_name": LLM_MODEL_NAME,
        "embedding_batch_num": 64,  # or another integer suitable for your setup
        "vector_db_storage_cls_kwargs": {
            "cosine_better_than_threshold": 0.2  # or another float threshold you want
        },
        "base_url": "https://api.openai.com/v1",
    }
    knowledge_graph = NetworkXStorage(namespace="nx_kg", global_config=faiss_global_config, embedding_func=openai_embed)  # Graph storage implementation
    entity_vector_db = FaissVectorDBStorage(namespace="faiss_entity", global_config=faiss_global_config, embedding_func=openai_embed)  # Vector storage for entities
    relationship_vector_db = FaissVectorDBStorage(namespace="faiss_relationship", global_config=faiss_global_config, embedding_func=openai_embed)  # Vector storage for relationships
    llm_cache = JsonKVStorage(namespace="llm_cache", global_config=kv_global_config, embedding_func=openai_embed)
    
    initialize_share_data()

    await knowledge_graph.initialize()
    await entity_vector_db.initialize()
    await relationship_vector_db.initialize()
    await llm_cache.initialize()  # if your cache supports/needs it

    # Step 4: Use the OpenAI LLM function from openai.py
    # Create a wrapper that matches the expected signature for extract_entities
    async def llm_wrapper(prompt, history_messages=None, max_tokens=None, **kwargs):
        if history_messages is None:
            history_messages = []
    

        # Use Google GenAI
        return await gemini_complete_if_cache(
            model=LLM_MODEL_NAME,
            prompt=prompt,
            history_messages=history_messages,
            hashing_kv=llm_cache,
            temperature=0.2,
            max_tokens=max_tokens or 1024,
        )
    
    # Step 5: Configure the extraction process
    global_config = {
        "llm_model_func": llm_wrapper,  # Use our wrapper function
        "llm_model_name": LLM_MODEL_NAME,  # Required for some OpenAI functions
        "entity_extract_max_gleaning": max_gleaning,
        "force_llm_summary_on_merge": False,
        "llm_model_max_token_size": 1024,
        "summary_to_max_tokens": 256,
        "tiktoken_model_name": LLM_MODEL_NAME,
        "addon_params": {
            "language": "English",
            "entity_types": [
                "Software or Computational Method",
                "Scientific Method",
                "Infrastructure or Equipment",
                "Institution or Organization",
                "Ecological or Biological Entity",
                "Pollutant or Material",
                "Time Expression",
                "Health or Disease Concept",
                "Food or Nutrition Element",
                "Demographic Group",
                "Measurement or Quantity"
            ]
        },
    }
    
    # Step 6: Optional status tracking
    pipeline_status = {
        "latest_message": "",
        "history_messages": []
    }
    pipeline_status_lock = asyncio.Lock()
    
    # Step 7: Run the extraction
    await extract_evidence(
        chunks=chunks,
        knowledge_graph_inst=knowledge_graph,
        entity_vdb=entity_vector_db,
        relationships_vdb=relationship_vector_db,
        global_config=global_config,
        seed_entities=seed_entities,
        pipeline_status=pipeline_status,
        pipeline_status_lock=pipeline_status_lock,
        llm_response_cache=llm_cache,
        cheatsheet_knowledge_graph_inst=cheatsheet_knowledge_graph_inst,
        write_result_to_txt=True,
        special_interest=CHEATSHEETS["special_interests"],
        prefix=prefix
    )
    
    return knowledge_graph


# Example usage in a Jupyter notebook
async def run_on_reference(content, nx_graph, seed_entities, max_gleaning, prefix=""):
    # monkeyReader = reader.MonkeyReader('monkey') # or 'lxml' or 'x2d'

    # read paper content
    # essay = monkeyReader.readEssay('./xmldata/Byjesh.pdf.tei.xml')
    # text = "\n".join(essay["Impact of fixed rise in temperature, CO 2 and change in rain fall"])
    text = content
    # nx_graph = nx.Graph()
    knowledge_graph = await process_text_to_knowledge_graph_on_reference(text,
                                                                         nx_graph,
                                                                         seed_entities,
                                                                         max_gleaning,
                                                                         "example.txt",
                                                                         prefix=prefix)
    # Now you can query the knowledge graph for entities and relationships
    
    return knowledge_graph

In [4]:
import networkx as nx
from utils import get_latest_result, read_knowledge_graph_from_pickle

async def get_othology_graph(entity_nodes, entity_edges):
    G = nx.Graph()
    for edge_keys, edge_list in entity_edges.items():
        for edge in edge_list:
            source = edge.get("src_id", "")
            target = edge.get("tgt_id", "")
            if source and target:
                if source in entity_nodes and target in entity_nodes:
                    G.add_edge(
                        entity_nodes[source][0]["entity_type"],
                        entity_nodes[target][0]["entity_type"],
                    )
    return G

In [ ]:
from utils import get_latest_result_by_prefix
from tqdm import trange

test_data = dataset_loader["subtask1_test"]
for index in trange(801, len(test_data)):
    reference_content = test_data.at[index, "reference"]
    claim_nodes, claim_edges = await get_latest_result_by_prefix(f"claim_test_{index}_result")
    claim_ontology_graph = await get_othology_graph(claim_nodes, claim_edges)
    seed_entities = list(claim_nodes.keys())

    _ = await run_on_reference(reference_content,
                               claim_ontology_graph,
                               seed_entities, 0, prefix=f"reference_test_{index}")

  0%|          | 0/199 [00:00<?, ?it/s]No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 248969 Shared-Data created for Single Process
INFO: Process 248969 initialized updated flags for namespace: [nx_kg]
INFO: Process 248969 initialized updated flags for namespace: [faiss_entity]
INFO: Process 248969 initialized updated flags for namespace: [faiss_relationship]
INFO: Process 248969 initialized updated flags for namespace: [llm_cache]
INFO: Process 248969 ready to initialize storage namespace: [llm_cache]


Loading latest result for prefix 'claim_test_801_result': claim_test_801_result_20250623_021818.pkl


/home/com3dian/Github/meta-knowledge-harvesting-llm/kaggle/operationCheatSheet.py:1231: RuntimeWarning: coroutine 'extract_evidence.<locals>.nightly_kg_to_text' was never awaited
  hint_prompt = fill_nightly_prompt.format(


Final result:
("entity"<|>"Myopia"<|>"Health or Disease Concept"<|>"Myopia is a significant public health problem, making it important to determine whether a bifocal spectacle treatment involving near prism slows myopia progression in children."<|>"Myopia is a significant public health problem"<|>[4]|>)##
("entity"<|>"Bifocal Spectacle Treatment"<|>"Infrastructure or Equipment"<|>"To determine whether bifocal and prismatic bifocal spectacles control myopia in children with high rates of myopia progression and to assess whether the treatment effect is dependent on the lag of accommodation and/or near phoria status."<|>"a bifocal spectacle treatment involving near prism"<|>[4]|>)##
("entity"<|>"Chinese-Canadian Children"<|>"Demographic Group"<|>"A total of 135 (73 female and 62 male) Chinese-Canadian children (aged 8-13 years; mean [SE] age, 10.29 [0.15] years; mean [SE] myopia, -3.08 [0.10] D) with myopia progression of at least 0.50 D in the preceding year were randomly assigned to 1 o

  1%|          | 1/199 [03:01<9:57:30, 181.06s/it]No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 248969 Shared-Data already initialized (multiprocess=False)
INFO: Process 248969 storage namespace already initialized: [llm_cache]


...

Final result:
("entity"<|>"Ischemic Stroke"<|>"Ischemic Stroke"<|>"Health or Disease Concept"<|>"Ischemic stroke is a leading cause of death and disability in the United States."<|>"Ischemic stroke"<|>"[2]"<|>"")##
("entity"<|>"Males"<|>"Males"<|>"Demographic Group"<|>"It is known that males and females respond differently to stroke."<|>"males"<|>"[2]"<|>"")##
("entity"<|>"Females"<|>"Females"<|>"Demographic Group"<|>"It is known that males and females respond differently to stroke."<|>"females"<|>"[2]"<|>"")##
("entity"<|>"Stroke"<|>"Stroke"<|>"Health or Disease Concept"<|>"It is known that males and females respond differently to stroke."<|>"stroke"<|>"[2]"<|>"")##
("entity"<|>"Stroke Outcome"<|>"Stroke Outcome"<|>"Health or Disease Concept"<|>"Females generally have strokes at older ages than males and, therefore, have a worse stroke outcome."<|>"worse stroke outcome"<|>"[2]"<|>"")##
("entity"<|>"Immune Response"<|>"Immune Response"<|>"Health or Disease Concept"<|>"Immune response i

  2%|▏         | 3/199 [06:38<6:32:04, 120.02s/it]No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 248969 Shared-Data already initialized (multiprocess=False)
INFO: Process 248969 storage namespace already initialized: [llm_cache]


Final result:
("entity"<|>"Subsistence Farmers"<|>"Demographic Group"<|>"Subsistence farmers in sub-Saharan Africa are highly vulnerable to food insecurity given their low adaptive capacity against ecological and socio-economic shocks."<|>"Subsistence farmers in sub-Saharan Africa"<|>"N/A"<|>"N/A")##
("entity"<|>"Food Insecurity"<|>"Health or Disease Concept"<|>"Subsistence farmers in sub-Saharan Africa are highly vulnerable to food insecurity given their low adaptive capacity against ecological and socio-economic shocks."<|>"food insecurity"<|>"N/A"<|>"N/A")##
("entity"<|>"Food Security"<|>"Food or Nutrition Element"<|>"Therefore, food security is one of their main challenges. Participatory action research across food value chains (FVCs) can help stabilize and enhance food security by developing upgrading strategies (UPS) that enhance specific aspects of crop production, post-harvest processing, marketing, income generation, and consumption."<|>"food security"<|>"N/A"<|>"N/A")##
("ent

  2%|▏         | 4/199 [08:34<6:25:03, 118.48s/it]No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 248969 Shared-Data already initialized (multiprocess=False)
INFO: Process 248969 storage namespace already initialized: [llm_cache]


Final result:
("entity"<|>"Laparoscopy"<|>"Scientific Method"<|>"Laparoscopy is the process of inspecting the abdominal cavity through an endoscope.")##
("entity"<|>"Carbon Dioxide"<|>"Pollutant or Material"<|>"Carbon dioxide is most universally used to insufflate the abdominal cavity to facilitate the view.")##
("entity"<|>"Pathophysiological Changes"<|>"Health or Disease Concept"<|>"However, several pathophysiological changes occur after carbon dioxide pneumoperitoneum and extremes of patient positioning.")##
("entity"<|>"Carbon Dioxide Pneumoperitoneum"<|>"Health or Disease Concept"<|>"However, several pathophysiological changes occur after carbon dioxide pneumoperitoneum and extremes of patient positioning.")##
("entity"<|>"Anesthetic Care"<|>"Scientific Method"<|>"A thorough understanding of these pathophysiological changes is fundamental for optimal anesthetic care.")##
("entity"<|>"Equipment"<|>"Infrastructure or Equipment"<|>"Because expertise and equipment have improved, lapar

In [5]:
import asyncio
from utils import get_latest_result_by_prefix
from tqdm import tqdm
import os
import asyncio
import os
import networkx as nx

from dotenv import load_dotenv
from operationCheatSheet import chunking_by_token_size, extract_entities, extract_evidence
from utils import compute_mdhash_id
from geminillm import gemini_complete_if_cache
from openaillm import openai_embed, openai_complete
from faiss_impl import FaissVectorDBStorage
from networkx_impl import NetworkXStorage
from json_kv_iml import JsonKVStorage
from shared_storage import initialize_share_data
from cheatsheet import CHEATSHEETS

test_data = dataset_loader["subtask1_test"]

import uuid
sem = asyncio.Semaphore(5)

async def process_one(index):
    async with sem:
        try:
            reference_content = test_data.at[index, "reference"]
            claim_nodes, claim_edges = await get_latest_result_by_prefix(f"claim_test_{index}_result")
            claim_ontology_graph = await get_othology_graph(claim_nodes, claim_edges)
            seed_entities = list(claim_nodes.keys())

            # Unique prefix and absolute output path
            prefix = f"reference_test_{index}_{uuid.uuid4().hex}"
            await run_on_reference(reference_content, claim_ontology_graph, seed_entities, 0, prefix=prefix)

        except Exception as e:
            print(f"[ERROR] Index {index}: {e}")

async def main():
    tasks = [process_one(i) for i in range(801, len(test_data))]
    for f in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
        await f

# Kick off the async run
await main()



  0%|          | 0/199 [00:00<?, ?it/s]

Loading latest result for prefix 'claim_test_817_result': claim_test_817_result_20250623_022457.pkl


No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 249448 Shared-Data created for Single Process
INFO: Process 249448 initialized updated flags for namespace: [nx_kg]
INFO: Process 249448 initialized updated flags for namespace: [faiss_entity]
INFO: Process 249448 initialized updated flags for namespace: [faiss_relationship]
INFO: Process 249448 initialized updated flags for namespace: [llm_cache]
INFO: Process 249448 ready to initialize storage namespace: [llm_cache]
No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 249448 Shared-Data already initialized (multiprocess=False)
INFO: Process 249448 storage namespace already initialized: [llm_cache]
No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 249448 Shared-Data already initialized (multiprocess=False)
INFO: Process 249448 storage nam

Loading latest result for prefix 'claim_test_847_result': claim_test_847_result_20250623_023915.pkl
Loading latest result for prefix 'claim_test_903_result': claim_test_903_result_20250623_030709.pkl
Loading latest result for prefix 'claim_test_959_result': claim_test_959_result_20250623_034007.pkl
Loading latest result for prefix 'claim_test_825_result': claim_test_825_result_20250623_022823.pkl


/home/com3dian/Github/meta-knowledge-harvesting-llm/kaggle/operationCheatSheet.py:1231: RuntimeWarning: coroutine 'extract_evidence.<locals>.nightly_kg_to_text' was never awaited
  hint_prompt = fill_nightly_prompt.format(


...

Final result:
("entity"<|>"Electric Vehicles"<|>"Electric Vehicles"<|>"Infrastructure or Equipment"<|>"In recent years electric mobility has gained a great deal of attention, leading to electric vehicles on the market and development of necessary charging infrastructure."<|>"electric vehicles"<|>"[12]"<|>"")##
("entity"<|>"Charging Infrastructure"<|>"Charging Infrastructure"<|>"Infrastructure or Equipment"<|>"Charging infrastructure is mostly enabled through subsidies by local or national governments to overcome the chicken and egg problem, while the business case for charge stations in this early stage of development is not yet sufficient."<|>"charging infrastructure"<|>"[12]"<|>"")##
("entity"<|>"Local or National Governments"<|>"Local or National Governments"<|>"Institution or Organization"<|>"Charging infrastructure is mostly enabled through subsidies by local or national governments to overcome the chicken and egg problem, while the business case for charge stations in this early 

 96%|█████████▌| 191/199 [5:28:24<13:29, 101.14s/it]

Final result:
("entity"<|>"lotus engine simulation program"<|>"Lotus Engine Simulation Program"<|>"Software or Computational Method"<|>"This paper investigates the use of alternative gaseous fuels in port fuel injection spark ignition engines via the Lotus engine simulation program."<|>"Lotus engine simulation program"<|>"[2]"<|>"")##
("entity"<|>"port fuel injection spark ignition engines"<|>"Port Fuel Injection Spark Ignition Engines"<|>"Infrastructure or Equipment"<|>"This paper investigates the use of alternative gaseous fuels in port fuel injection spark ignition engines via the Lotus engine simulation program. To this end, the engine model for a port fuel injection spark ignition engine was designed, and engine performance was determined separately for gasoline and alternative gaseous fuels, i.e. compressed natural gas and liquefied petroleum gas."<|>"port fuel injection spark ignition engines"<|>"[2]"<|>"")##
("entity"<|>"alternative gaseous fuels"<|>"Alternative Gaseous Fuels"<

100%|██████████| 199/199 [5:32:26<00:00, 100.23s/it]

Final result:
("entity"<|>"Skipjack Tuna"<|>"Skipjack Tuna"<|>"Ecological or Biological Entity"<|>"Skipjack tuna habitat in the western North Pacific was studied from satellite remotely sensed environment and catch data, using generalized additive models and geographic information systems."<|>"Skipjack tuna"<|>"1"<|>"N/A")##
("entity"<|>"Generalized Additive Models"<|>"Generalized Additive Models"<|>"Software or Computational Method"<|>"Fifteen generalized additive models were constructed with skipjack catch per unit effort as a response variable, and sea surface temperature, sea surface height anomalies and eddy kinetic energy as model covariates to assess the effect of environment on catch per unit effort (skipjack tuna abundance)."<|>"generalized additive models"<|>"1"<|>"N/A")##
("entity"<|>"Geographic Information Systems"<|>"Geographic Information Systems"<|>"Software or Computational Method"<|>"Skipjack tuna habitat in the western North Pacific was studied from satellite remotely

# Classify base on the evidence

In [1]:
def write_prediction_to_file(row_id, prediction, output_file="outputs/response_test.txt"):
    """
    Write a line with row ID and prediction to the output file
    
    Args:
        row_id: The ID/index of the row being processed
        prediction: The prediction result
        output_file: Path to the output file
    """
    # Ensure outputs directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Write the prediction with row ID to file
    with open(output_file, 'a', encoding='utf-8') as f:
        f.write(f"{row_id}\t{prediction}\n")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import numpy as np
from kg_compare import get_src_tgt_dict, get_map_src_tgt_dict, find_best_span

from kg_compare import compare_knowledge_graphs_similarity, find_best_span_for_all_evidences
import seaborn as sns

import numpy as np
from sklearn.cluster import KMeans
all_responses = []

output_file = "outputs/response_test.txt"
os.makedirs(os.path.dirname(output_file), exist_ok=True)


for index in range(len(test_data)):
    if index % 50 == 0:
        print(f"Processing index {index}")
    
    row_id = test_data.at[index, "ID"]

    claim_nodes, claim_edges = get_latest_result_by_prefix(f"claim_test_{index}_")
    reference_nodes, reference_edges = get_latest_result_by_prefix(f"reference_test_{index}_")
    reference_content = test_data.at[index, "reference"]

    claim_node_list = list(claim_nodes.keys())
    reference_node_list = list(reference_nodes.keys())

    if claim_node_list == [] or reference_node_list == []:
        all_responses.append("Unverifiable")
        write_prediction_to_file(row_id, "Unverifiable")
        continue
        
    similarity_matrix = compare_knowledge_graphs_similarity(claim_node_list,
                                                            reference_node_list,
                                                            if_plot=False)
    src_tgt_dict = get_src_tgt_dict(similarity_matrix, claim_node_list, reference_node_list)

    claim_reference_edge_map = get_map_src_tgt_dict(src_tgt_dict, claim_edges)

    evidence_text = find_best_span_for_all_evidences(reference_edges,
                                 claim_reference_edge_map,
                                 reference_content)
    
    if evidence_text == "":
        all_responses.append("Unverifiable")
        write_prediction_to_file(row_id, "Unverifiable")
        continue

    classification_prompt = f"""
    You are an expert hallucination classifier. Your task is to classify a claim based solely on the provided evidence into one of the following three categories:

    - Entailment
    - Contradiction
    - Unverifiable

    Claim:
    {claim_content}

    Evidence:
    {evidence_text}

    Instructions:
    - You MUST respond with ONLY one of the three words: "Entailment", "Contradiction", or "Unverifiable"
    - Respond "Unverifiable" if the evidence is irrelevant, insufficient, or cannot confirm or deny the claim
    - Respond "Contradiction" if the claim contradicts, misinterprets, misleads, or overstates the evidence
    - Respond "Entailment" if the evidence reasonably supports the claim, even if some uncertainty exists
    - Do NOT provide explanations, justifications, or any additional text
    - Your entire response must be EXACTLY one of these three words: "Entailment", "Contradiction", or "Unverifiable"
    """

    response = await llm_wrapper(classification_prompt)
    all_responses.append(response)
    write_prediction_to_file(row_id, response)
    
    

In [ ]:
# Helper function to write predictions to file
import os

def write_prediction_to_file(row_id, prediction, output_file="outputs/response_test.txt"):
    """
    Write a line with row ID and prediction to the output file
    
    Args:
        row_id: The ID/index of the row being processed
        prediction: The prediction result
        output_file: Path to the output file
    """
    # Ensure outputs directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Write the prediction with row ID to file
    with open(output_file, 'a', encoding='utf-8') as f:
        f.write(f"{row_id}\t{prediction}\n")
    
    print(f"Written prediction for row {row_id}: {prediction}")

# Example usage in your prediction loop:
# Replace your existing all_responses.append() calls with:
# 
# For "Unverifiable" predictions:
# prediction = "Unverifiable"
# all_responses.append(prediction)
# write_prediction_to_file(row_id, prediction)  # Add this line
#
# For regular predictions:
# prediction = response  # your response/prediction
# all_responses.append(prediction)
# write_prediction_to_file(row_id, prediction)  # Add this line


In [ ]:
# Clear the output file at the start of processing
output_file = "outputs/response_test.txt"
if os.path.exists(output_file):
    os.remove(output_file)
    print(f"Cleared existing {output_file}")

# Add header to the file
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("row_id\tprediction\n")
print(f"Initialized {output_file} with header")
